In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import plotly.graph_objects as go
from datetime import datetime

In [ ]:
# Load data
df = pd.read_excel('ML_LC_DEST_Refined_V1.xlsx')

# Extract date from TRAIN_ID
df['DATE'] = pd.to_datetime(df['TD'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0])

# Split into train and test
split_date = pd.to_datetime('2024-01-01')
train_df = df[df['DATE'] < split_date].copy()
test_df = df[df['DATE'] >= split_date].copy()

print(f"Original dataset: {df['TRAIN_ID'].nunique()} trains")
print(f"Train set (before 2024): {train_df['TRAIN_ID'].nunique()} trains")
print(f"Test set (2024+): {test_df['TRAIN_ID'].nunique()} trains")

Original dataset: 693 trains
Train set (before 2024): 543 trains
Test set (2024+): 150 trains


In [ ]:
# Define bin ranges for DWELL_TIME to prepare for outlier detection
# Create bins with varying sizes: 0-10 (0.5 intervals), 10-20 (1.0 intervals), 20+ (5.0 intervals)1

# Define bins
bins = np.concatenate([
    np.arange(0, 10.5, 0.5),    # 0-10: bins of 0.5
    np.arange(11, 21, 1),       # 10-20: bins of 1.0
    np.arange(25, 145, 5)       # 20-140: bins of 5.0
])

# Create DataFrame with interval objects
bins_df = pd.DataFrame({
    'BIN_RANGE': pd.IntervalIndex.from_breaks(bins, closed='right')
})

print(f"Number of bins created: {len(bins_df)}")
bins_df.head(10)

Number of bins created: 54


,BIN_RANGE
0,"(0.0, 0.5]"
1,"(0.5, 1.0]"
2,"(1.0, 1.5]"
3,"(1.5, 2.0]"
4,"(2.0, 2.5]"
5,"(2.5, 3.0]"
6,"(3.0, 3.5]"
7,"(3.5, 4.0]"
8,"(4.0, 4.5]"
9,"(4.5, 5.0]"


In [ ]:
# Map each train ID and its dwell time to the appropriate bin range
# This will add a new column showing which bin each record belongs to in both the train and test datasets

# Assign bins to train and test datasets
train_df['DWELL_TIME_BIN'] = pd.cut(train_df['DWELL_TIME'], bins=bins)
test_df['DWELL_TIME_BIN'] = pd.cut(test_df['DWELL_TIME'], bins=bins)
df['DWELL_TIME_BIN'] = pd.cut(df['DWELL_TIME'], bins=bins)


# Show sample from training data
print("Sample from training data:")
print(train_df[['TRAIN_ID', 'DWELL_TIME', 'DWELL_TIME_BIN']].head(10))

Sample from training data:
                 TRAIN_ID  DWELL_TIME DWELL_TIME_BIN
0   XBERBSB928H2021-05-28       0.300     (0.0, 0.5]
2   XCROBSB921A2023-09-21       0.933     (0.5, 1.0]
3   XCROEGE922A2023-09-22       1.017     (1.0, 1.5]
4   XCROHBG919A2023-09-19       1.400     (1.0, 1.5]
5   XPOWBNY925A2021-09-25       0.983     (0.5, 1.0]
6   XPOWBRE920A2021-09-20       2.100     (2.0, 2.5]
7   XPOWBSB901A2021-10-01       2.150     (2.0, 2.5]
8   XPOWEGE902A2021-10-02       0.850     (0.5, 1.0]
9   XPOWGWN903A2021-10-03       4.433     (4.0, 4.5]
10  XPOWCSN921A2021-09-21       0.967     (0.5, 1.0]


In [ ]:
# Count unique trains per bin using TRAINING data and add to bins_df
train_counts_by_bin = train_df.groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()
bins_df['NB_OF_TRAIN_TRAINS'] = bins_df['BIN_RANGE'].map(train_counts_by_bin).fillna(0).astype(int)

print(f"Total trains counted from TRAINING set: {bins_df['NB_OF_TRAIN_TRAINS'].sum()}")
print(bins_df.sort_values(by='NB_OF_TRAIN_TRAINS', ascending=False))

Total trains counted from TRAINING set: 543
         BIN_RANGE  NB_OF_TRAIN_TRAINS
1       (0.5, 1.0]                 109
2       (1.0, 1.5]                  77
3       (1.5, 2.0]                  48
4       (2.0, 2.5]                  47
5       (2.5, 3.0]                  37
6       (3.0, 3.5]                  27
7       (3.5, 4.0]                  24
8       (4.0, 4.5]                  23
10      (5.0, 5.5]                  19
9       (4.5, 5.0]                  15
20    (10.0, 11.0]                  13
13      (6.5, 7.0]                  11
12      (6.0, 6.5]                   9
11      (5.5, 6.0]                   9
19     (9.5, 10.0]                   9
14      (7.0, 7.5]                   8
15      (7.5, 8.0]                   8
16      (8.0, 8.5]                   7
0       (0.0, 0.5]                   6
23    (13.0, 14.0]                   6
24    (14.0, 15.0]                   5
17      (8.5, 9.0]                   4
21    (11.0, 12.0]                   4
18      (9.0, 9.5]  

In [ ]:
# Filter bins_df to keep only bins with more than 0 trains
bins_df = bins_df[bins_df['NB_OF_TRAIN_TRAINS'] > 0]  # CORRECT column name
bins_df

,BIN_RANGE,NB_OF_TRAIN_TRAINS
0,"(0.0, 0.5]",6
1,"(0.5, 1.0]",109
2,"(1.0, 1.5]",77
3,"(1.5, 2.0]",48
4,"(2.0, 2.5]",47
5,"(2.5, 3.0]",37
6,"(3.0, 3.5]",27
7,"(3.5, 4.0]",24
8,"(4.0, 4.5]",23
9,"(4.5, 5.0]",15


In [ ]:
# Find bins containing at least 5% of trains
threshold = int(np.ceil(df['TRAIN_ID'].nunique() * 0.05))
bins_above_threshold = bins_df[bins_df['NB_OF_TRAIN_TRAINS'] >= threshold]

print(f"5% threshold: {threshold} trains")
print(f"Bins meeting threshold: {bins_above_threshold.shape[0]}")
bins_above_threshold


5% threshold: 35 trains
Bins meeting threshold: 5


,BIN_RANGE,NB_OF_TRAIN_TRAINS
1,"(0.5, 1.0]",109
2,"(1.0, 1.5]",77
3,"(1.5, 2.0]",48
4,"(2.0, 2.5]",47
5,"(2.5, 3.0]",37


In [ ]:
# Get bounds from first and last bin above threshold
lower_bound = bins_above_threshold.iloc[0]['BIN_RANGE'].left
upper_bound = bins_above_threshold.iloc[-1]['BIN_RANGE'].right

# Create summary
summary_df = pd.DataFrame({
    'STN_333': ['MINOT'],
    'Overall_Count': [df['TRAIN_ID'].nunique()],  # Total unique trains in dataset
    'Lower_Bound': [lower_bound],
    'Upper_Bound': [upper_bound]
})

summary_df

,STN_333,Overall_Count,Lower_Bound,Upper_Bound
0,MINOT,693,0.5,3.0


In [ ]:
# Get outlier bounds from the bins that meet threshold
lower_bound = bins_above_threshold.iloc[0]['BIN_RANGE'].left
upper_bound = bins_above_threshold.iloc[-1]['BIN_RANGE'].right

print(f"Outlier bounds: {lower_bound} to {upper_bound}")

# Add outlier flag to all datasets
train_df['IS_OUTLIER'] = (train_df['DWELL_TIME'] < lower_bound) | (train_df['DWELL_TIME'] > upper_bound)
test_df['IS_OUTLIER'] = (test_df['DWELL_TIME'] < lower_bound) | (test_df['DWELL_TIME'] > upper_bound)
df['IS_OUTLIER'] = (df['DWELL_TIME'] < lower_bound) | (df['DWELL_TIME'] > upper_bound)

# Quick summary
print(f"\nTraining outliers: {train_df['IS_OUTLIER'].sum()} out of {len(train_df)} records")
print(f"Test outliers: {test_df['IS_OUTLIER'].sum()} out of {len(test_df)} records")

# Show sample
print("\nSample:")
df[['TRAIN_ID', 'DWELL_TIME','IS_OUTLIER']].head(5)

Outlier bounds: 0.5 to 3.0

Training outliers: 225 out of 543 records
Test outliers: 65 out of 150 records

Sample:


,TRAIN_ID,DWELL_TIME,IS_OUTLIER
0,XBERBSB928H2021-05-28,0.300,True
1,XKAHKND911H2024-08-11,3.417,True
2,XCROBSB921A2023-09-21,0.933,False
3,XCROEGE922A2023-09-22,1.017,False
4,XCROHBG919A2023-09-19,1.400,False


In [ ]:
processed_df = pd.concat([train_df, test_df])
processed_df.shape

(693, 24)

In [ ]:
processed_df.head()

,TRAIN_ID,STN_333,STN_ST,STN_TYPE_CD,STN_SEQ_NBR,TRN_MILES_TOT_TD,crew_order,DPT_DIR,REQ_INSP,case,...,TRN_MILES_TOT_DEST,STN_SEQ_NBR_DEST,DISTANCE,LAST_CREW_STATION,SECOND_LAST_CREW_STATION,TRAVEL_TIME,DWELL_TIME,DATE,DWELL_TIME_BIN,IS_OUTLIER
0,XBERBSB928H2021-05-28,MINOT,ND,C,40,22.7,1,E,N,General,...,149.4,210,126.7,MINOT,GLASGOW,5.750,0.300,2021-05-30,"(0.0, 0.5]",True
2,XCROBSB921A2023-09-21,MINOT,ND,C,90,63.6,1,E,N,General,...,190.3,260,126.7,MINOT,GLASGOW,5.750,0.933,2023-09-21,"(0.5, 1.0]",False
3,XCROEGE922A2023-09-22,MINOT,ND,C,90,63.6,1,E,N,General,...,281.0,370,217.4,MINOT,GLASGOW,5.500,1.017,2023-09-22,"(1.0, 1.5]",False
4,XCROHBG919A2023-09-19,MINOT,ND,C,90,63.6,1,E,N,General,...,154.9,210,91.3,MINOT,GLASGOW,1.750,1.400,2023-09-19,"(1.0, 1.5]",False
5,XPOWBNY925A2021-09-25,MINOT,ND,C,110,80.0,1,E,N,General,...,297.4,390,217.4,MINOT,GLASGOW,8.917,0.983,2021-09-25,"(0.5, 1.0]",False


In [ ]:
processed_df['IS_OUTLIER'].unique()

array([ True, False])

In [ ]:
processed_df[processed_df['IS_OUTLIER']==False]['TRAIN_ID'].nunique()

403

In [ ]:
processed_df.columns

Index(['TRAIN_ID', 'STN_333', 'STN_ST', 'STN_TYPE_CD', 'STN_SEQ_NBR',
       'TRN_MILES_TOT_TD', 'crew_order', 'DPT_DIR', 'REQ_INSP', 'case', 'TA',
       'TD', 'DESTINATION', 'ArrivalDestination', 'TRN_MILES_TOT_DEST',
       'STN_SEQ_NBR_DEST', 'DISTANCE', 'LAST_CREW_STATION',
       'SECOND_LAST_CREW_STATION', 'TRAVEL_TIME', 'DWELL_TIME', 'DATE',
       'DWELL_TIME_BIN', 'IS_OUTLIER'],
      dtype='object')

In [ ]:
# Get date range for TA
min_date = pd.to_datetime(processed_df['TA']).min()
max_date = pd.to_datetime(processed_df['TA']).max()
print(f"Date range: {min_date.date()} to {max_date.date()}")

Date range: 2020-05-05 to 2024-08-20


C:\Users\rita03.TRN\AppData\Local\Temp\ipykernel_26752\552750185.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  min_date = pd.to_datetime(processed_df['TA']).min()
C:\Users\rita03.TRN\AppData\Local\Temp\ipykernel_26752\552750185.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  max_date = pd.to_datetime(processed_df['TA']).max()


In [ ]:
# Count unique trains per bin for test and all datasets
test_counts = test_df.groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()
all_counts = df.groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()

# Map counts to bins_df
bins_df['NB_TRAINS_TEST'] = bins_df['BIN_RANGE'].map(test_counts).fillna(0).astype(int)
bins_df['NB_TRAINS_ALL'] = bins_df['BIN_RANGE'].map(all_counts).fillna(0).astype(int)

# Keep only bins with data
bins_df = bins_df[bins_df['NB_TRAINS_ALL'] > 0]

# Display result
bins_df

,BIN_RANGE,NB_OF_TRAIN_TRAINS,NB_TRAINS_TEST,NB_TRAINS_ALL
0,"(0.0, 0.5]",6,1,7
1,"(0.5, 1.0]",109,27,136
2,"(1.0, 1.5]",77,30,107
3,"(1.5, 2.0]",48,10,58
4,"(2.0, 2.5]",47,9,56
5,"(2.5, 3.0]",37,9,46
6,"(3.0, 3.5]",27,12,39
7,"(3.5, 4.0]",24,4,28
8,"(4.0, 4.5]",23,4,27
9,"(4.5, 5.0]",15,6,21


In [ ]:
# Create figure
fig = go.Figure()

# Add trace for with outliers (using NB_TRAINS_ALL)
fig.add_trace(go.Bar(
    name='DWELL_TIME_with_outliers',
    x=[str(interval) for interval in bins_df['BIN_RANGE']],
    y=bins_df['NB_TRAINS_ALL'],
    marker_color='blue',
    opacity=0.7,
    text=bins_df['NB_TRAINS_ALL'],
    textposition='auto'
))

# Count unique trains without outliers
bin_counts_without = df[df['IS_OUTLIER'] == False].groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()
bins_df['NB_TRAINS_NO_OUTLIERS'] = bins_df['BIN_RANGE'].map(bin_counts_without).fillna(0).astype(int)

# Add trace for without outliers
fig.add_trace(go.Bar(
    name='DWELL_TIME_without_outliers',
    x=[str(interval) for interval in bins_df['BIN_RANGE']],
    y=bins_df['NB_TRAINS_NO_OUTLIERS'],
    marker_color='red',
    opacity=0.7,
    text=bins_df['NB_TRAINS_NO_OUTLIERS'],
    textposition='auto'
))

# Update layout
fig.update_layout(
    title='Distribution plot for DWELL_TIME with & Without outliers - ALL DESTINATIONS',
    xaxis_title='DWELL_TIME_BIN',
    yaxis_title='Count of Train ID',
    barmode='group',
    xaxis_tickangle=-45,
    height=600,
    width=1200,
    hovermode='x unified'
)

# Show plot
fig.show()


In [ ]:
# Function to plot for any destination
def plot_destination(destination_name):

    # Filter data for destination
    dest_data = df[df['DESTINATION'] == destination_name]

    # Count total trains for this destination
    total_trains_with = dest_data['TRAIN_ID'].nunique()
    total_trains_without = dest_data[dest_data['IS_OUTLIER'] == False]['TRAIN_ID'].nunique()

    # Count per bin
    counts_with = dest_data.groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()
    counts_without = dest_data[dest_data['IS_OUTLIER'] == False].groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()

    # Get bins that have data for this destination (for plotting)
    bins_with_data = sorted(set(counts_with[counts_with > 0].index) | set(counts_without[counts_without > 0].index))

    # Create figure
    fig = go.Figure()

    # Add bars for with outliers
    fig.add_trace(go.Bar(
        name='DWELL_TIME_with_outliers',
        x=[str(interval) for interval in bins_with_data],
        y=[counts_with.get(bin, 0) for bin in bins_with_data],
        marker_color='blue',
        opacity=0.7,
        text=[counts_with.get(bin, 0) for bin in bins_with_data],
        textposition='auto'
    ))

    # Add bars for without outliers
    fig.add_trace(go.Bar(
        name='DWELL_TIME_without_outliers',
        x=[str(interval) for interval in bins_with_data],
        y=[counts_without.get(bin, 0) for bin in bins_with_data],
        marker_color='red',
        opacity=0.7,
        text=[counts_without.get(bin, 0) for bin in bins_with_data],
        textposition='auto'
    ))

    # Update layout
    fig.update_layout(
        title=f'Distribution plot for DWELL_TIME with & Without outliers for {destination_name}',
        xaxis_title='DWELL_TIME_BIN',
        yaxis_title='Count of Train ID',
        barmode='group',
        xaxis_tickangle=-45,
        height=600,
        width=1000,
        hovermode='x unified'
    )

    fig.show()

# Plot for each destination
destinations = ['BISBEE', 'CASSELTON', 'HAMBERG', 'PILLSBURY', 'FARGO']
for dest in destinations:
    plot_destination(dest)

In [ ]:
# Add outlier detection to dataframe based on dwell time bins.
#Parameters:
    #Input dataframe with columns including 'TRAIN_ID', 'DWELL_TIME', and 'TD'
    #Output : Same dataframe with added columns 'DWELL_TIME_BIN' and 'IS_OUTLIER'

def outlier_detection(df):

    # Extract date and split data
    df['DATE'] = pd.to_datetime(df['TD'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0])
    split_date = pd.to_datetime('2024-01-01')
    train_df = df[df['DATE'] < split_date].copy()

    # Define bins
    bins = np.concatenate([
        np.arange(0, 10.5, 0.5),
        np.arange(11, 21, 1),
        np.arange(25, 145, 5)
    ])

    # Assign bins
    df['DWELL_TIME_BIN'] = pd.cut(df['DWELL_TIME'], bins=bins)
    train_df['DWELL_TIME_BIN'] = pd.cut(train_df['DWELL_TIME'], bins=bins)

    # Create bins df and count trains
    bins_df = pd.DataFrame({'BIN_RANGE': pd.IntervalIndex.from_breaks(bins, closed='right')})
    train_counts = train_df.groupby('DWELL_TIME_BIN', observed=False)['TRAIN_ID'].nunique()
    bins_df['NB_TRAINS'] = bins_df['BIN_RANGE'].map(train_counts).fillna(0).astype(int)

    # Find threshold and bounds
    threshold = int(np.ceil(train_df['TRAIN_ID'].nunique() * 0.05))
    bins_above_threshold = bins_df[bins_df['NB_TRAINS'] >= threshold]
    lower_bound = bins_above_threshold.iloc[0]['BIN_RANGE'].left
    upper_bound = bins_above_threshold.iloc[-1]['BIN_RANGE'].right

    # Add outlier flag
    df['IS_OUTLIER'] = (df['DWELL_TIME'] < lower_bound) | (df['DWELL_TIME'] > upper_bound)

    # Drop temporary DATE column created earlier to split the data in train and test
    df = df.drop('DATE', axis=1)

    return df

outlier_detection(df)

,TRAIN_ID,STN_333,STN_ST,STN_TYPE_CD,STN_SEQ_NBR,TRN_MILES_TOT_TD,crew_order,DPT_DIR,REQ_INSP,case,...,ArrivalDestination,TRN_MILES_TOT_DEST,STN_SEQ_NBR_DEST,DISTANCE,LAST_CREW_STATION,SECOND_LAST_CREW_STATION,TRAVEL_TIME,DWELL_TIME,DWELL_TIME_BIN,IS_OUTLIER
0,XBERBSB928H2021-05-28,MINOT,ND,C,40,22.7,1,E,N,General,...,2021-05-30T11:07:00-05:00,149.4,210,126.7,MINOT,GLASGOW,5.750,0.300,"(0.0, 0.5]",True
1,XKAHKND911H2024-08-11,MINOT,ND,C,50,31.0,1,E,N,General,...,2024-08-12T16:07:00-05:00,248.4,330,217.4,MINOT,GLASGOW,8.400,3.417,"(3.0, 3.5]",True
2,XCROBSB921A2023-09-21,MINOT,ND,C,90,63.6,1,E,N,General,...,2023-09-22T02:34:00-05:00,190.3,260,126.7,MINOT,GLASGOW,5.750,0.933,"(0.5, 1.0]",False
3,XCROEGE922A2023-09-22,MINOT,ND,C,90,63.6,1,E,N,General,...,2023-09-23T01:07:00-05:00,281.0,370,217.4,MINOT,GLASGOW,5.500,1.017,"(1.0, 1.5]",False
4,XCROHBG919A2023-09-19,MINOT,ND,C,90,63.6,1,E,N,General,...,2023-09-19T19:30:00-05:00,154.9,210,91.3,MINOT,GLASGOW,1.750,1.400,"(1.0, 1.5]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,XVAWBRE905H2021-09-05,MINOT,ND,C,2420,1465.3,1,E,N,General,...,2021-09-09T15:51:00-05:00,1682.7,2700,217.4,MINOT,GLASGOW,7.433,2.050,"(2.0, 2.5]",False
689,XRGTCSN905H2024-08-05,MINOT,ND,C,2460,1468.0,1,E,N,General,...,2024-08-11T04:22:00-05:00,1685.4,2740,217.4,MINOT,GLASGOW,6.200,1.417,"(1.0, 1.5]",False
690,XRGTHBG927H2021-03-27,MINOT,ND,C,2460,1468.0,1,E,N,General,...,2021-04-03T00:56:00-05:00,1559.3,2580,91.3,MINOT,GLASGOW,3.100,1.383,"(1.0, 1.5]",False
691,XRGTHBG926H2020-11-26,MINOT,ND,C,2460,1468.0,1,E,N,General,...,2020-11-29T07:41:00-06:00,1559.3,2580,91.3,MINOT,GLASGOW,1.767,2.183,"(2.0, 2.5]",False
